In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
table_list = ['Students', 'Subjects', 'Examinations']
for table in table_list:
    display_table(table)

+------------+--------------+
| student_id | student_name |
+------------+--------------+
|     1      |    Alice     |
|     2      |     Bob      |
|     13     |     John     |
|     6      |     Alex     |
+------------+--------------+
+--------------+
| subject_name |
+--------------+
|     Math     |
|   Physics    |
| Programming  |
+--------------+
+------------+--------------+
| student_id | subject_name |
+------------+--------------+
|     1      |     Math     |
|     1      |   Physics    |
|     1      | Programming  |
|     2      | Programming  |
|     1      |   Physics    |
|     1      |     Math     |
|     13     |     Math     |
|     13     | Programming  |
|     13     |   Physics    |
|     2      |     Math     |
|     1      |     Math     |
+------------+--------------+


### Write an SQL query to find the number of times each student attended each exam.Return the result table ordered by `student_id` and `subject_name`.

```
+------------+--------------+--------------+----------------+
| student_id | student_name | subject_name | attended_exams |
+------------+--------------+--------------+----------------+
| 1          | Alice        | Math         | 3              |
| 1          | Alice        | Physics      | 2              |
| 1          | Alice        | Programming  | 1              |
| 2          | Bob          | Math         | 1              |
| 2          | Bob          | Physics      | 0              |
| 2          | Bob          | Programming  | 1              |
| 6          | Alex         | Math         | 0              |
| 6          | Alex         | Physics      | 0              |
| 6          | Alex         | Programming  | 0              |
| 13         | John         | Math         | 1              |
| 13         | John         | Physics      | 1              |
| 13         | John         | Programming  | 1              |
+------------+--------------+--------------+----------------+
```

In [3]:
%%sql 

SELECT s.student_id, s.student_name, sub.subject_name
FROM Students s
CROSS JOIN Subjects sub

student_id,student_name,subject_name
1,Alice,Programming
1,Alice,Physics
1,Alice,Math
2,Bob,Programming
2,Bob,Physics
2,Bob,Math
13,John,Programming
13,John,Physics
13,John,Math
6,Alex,Programming


In [4]:
%%sql 

SELECT 
    s.student_id, 
    s.student_name, 
    sub.subject_name, 
    COUNT(e.student_id) AS attended_exams
FROM Students s
CROSS JOIN Subjects sub
LEFT JOIN Examinations e 
    ON s.student_id = e.student_id 
    AND sub.subject_name = e.subject_name
GROUP BY s.student_id, sub.subject_name, s.student_name
ORDER BY s.student_id, sub.subject_name;


student_id,student_name,subject_name,attended_exams
1,Alice,Math,3
1,Alice,Physics,2
1,Alice,Programming,1
2,Bob,Math,1
2,Bob,Physics,0
2,Bob,Programming,1
6,Alex,Math,0
6,Alex,Physics,0
6,Alex,Programming,0
13,John,Math,1


# Using Pandas

In [5]:
import pandas as pd

In [6]:
students_query = %sql SELECT * FROM Students # type: ignore 
exams_query = %sql SELECT * FROM Examinations # type: ignore 
subjects_query = %sql SELECT * FROM Subjects # type: ignore

students_df = students_query.DataFrame()
exams_df = exams_query.DataFrame()
subjects_df = subjects_query.DataFrame()

display(students_df, exams_df, subjects_df)

,student_id,student_name
0,1,Alice
1,2,Bob
2,13,John
3,6,Alex


,student_id,subject_name
0,1,Math
1,1,Physics
2,1,Programming
3,2,Programming
4,1,Physics
5,1,Math
6,13,Math
7,13,Programming
8,13,Physics
9,2,Math


,subject_name
0,Math
1,Physics
2,Programming


## join()

In [7]:
joined_df = students_df.join(subjects_df, how='cross')
joined_df

,student_id,student_name,subject_name
0,1,Alice,Math
1,1,Alice,Physics
2,1,Alice,Programming
3,2,Bob,Math
4,2,Bob,Physics
5,2,Bob,Programming
6,13,John,Math
7,13,John,Physics
8,13,John,Programming
9,6,Alex,Math


In [8]:
temp_df = exams_df.groupby(['student_id', 'subject_name']) \
    .agg(attended_exams=('student_id', 'count'))
temp_df

attended_exams
student_id subject_name                
1          Math                       3
           Physics                    2
           Programming                1
2          Math                       1
           Programming                1
13         Math                       1
           Physics                    1
           Programming                1

## INCOMPLETE -- NEED TO FIGURE OUT